In [ ]:
import requests
import time
import pandas as pd
import os

In [4]:
# 엑셀 파일 불러오기
df = pd.read_excel("C:/Users/hk522/Desktop/병원 풀데이터.xlsx", sheet_name="Sheet1")
#pharmacy_df = pd.read_excel("C:/Users/hk522/Desktop/약국 풀 데이터.xlsx")

In [5]:
# 설정
app_key = "h378OZ68lH4U8fIl3Htou9WCfyggLHLdaxj4HwIE"
address_col = 'dutyAddr'     # 변환 대상 열 이름

In [ ]:
# ---------- 1. 미처리 데이터 불러오기 ----------
df = pd.read_excel("미처리.xlsx")

In [6]:
# ---------- 2. 지번 주소 변환 함수 ----------
def convert_to_jibun(road_address, app_key):
    url = "http://apis.openapi.sk.com/tmap/geo/convertAddress"
    params = {
        "version": 1,
        "addr": road_address,
        "appKey": app_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    try:
        return data['addressInfo']['jibunAddr']
    except:
        return None

In [ ]:
# ---------- 3. 최대 2만건까지 처리 ----------
count = 0
max_count = 20000
unprocessed_idx = []

for i in range(len(df)):
    address = df.loc[i, address_col]
    if pd.isna(address):
        continue
    if count >= max_count:
        unprocessed_idx.append(i)
        continue

    jibun = convert_to_jibun(address, app_key)
    df.loc[i, address_col] = jibun
    count += 1
    time.sleep(0.2)  # 과도한 요청 방지

print(f"✅ 오늘 {count}건 처리 완료")

In [ ]:
# ---------- 4. 기존 파일 있으면 이어붙이고 없으면 새로 저장 ----------
processed_today = df.drop(index=unprocessed_idx)

if os.path.exists("C:/Users/hk522/Desktop/병원 풀데이터 지번 주소.xlsx"):
    processed_prev = pd.read_excel("C:/Users/hk522/Desktop/병원 풀데이터 지번 주소.xlsx")
    updated_df = pd.concat([processed_prev, processed_today], ignore_index=True)
else:
    updated_df = processed_today

updated_df.to_excel("C:/Users/hk522/Desktop/병원 풀데이터 지번 주소.xlsx", index=False)
print("📄 '병원 풀데이터 지번 주소.xlsx'에 저장 완료")

In [ ]:
# ---------- 5. 남은 미처리 저장 ----------
if unprocessed_idx:
    df.iloc[unprocessed_idx].to_excel("C:/Users/hk522/Desktop/미처리.xlsx", index=False)
    print(f"📂 {len(unprocessed_idx)}건은 '미처리.xlsx'로 저장됨 (내일 처리)")
else:
    print("🎉 전체 데이터 변환 완료! 더 이상 미처리 없음.")